# Детектор лиц
## RetinaFace: Single-shot Multi-level Face Localisation in the Wild 
![FRS](imgs/retinaface.png "FRS")

Реализация сетки взята из репозитория https://github.com/peteryuX/retinaface-tf2. Данная архитектура немного отходит от оригинальной бумаги и оптимизирует __3 компонентный loss__, вместо __4 компонентного__ как оригинальной статье - https://arxiv.org/pdf/1905.00641.pdf. Но так как мы обучаемся на датасете __Wider Faces__, то попиксельная регрессия лица для построения 3D сетки нам не нужна.
![](imgs/loss.png)
Рассматриваемая ахритектура является так называем Single-shot детектором или One-stage, когда за 1 проход сети в случае __RetinaFace__ мы получаем координаты баундинг бокса, метку класса и 5 точек лица.

In [1]:
import os
import zipfile

# Склонируем репозиторий
#!git clone https://github.com/peteryuX/retinaface-tf2.git retinaface

In [2]:
# Определим пути нашего приложения в контейнере
APP_DIR = '/app/retinaface/'
DATA_DIR = APP_DIR + 'data/'
DATASET_DIR = DATA_DIR + 'widerface' + '/'

# Путь к конфигурации обучения
CFG_PATH = f'{APP_DIR}configs/retinaface_res50.yaml' # retinaface_mbv2.yaml

import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                
                
def dataset_loader(gdrive_file_id=None, dataset_path=None, ds_type='train'):
    assert ds_type in ['train', 'val', 'annotations'], 'Датасет может быть test, val, annotations'
    assert gdrive_file_id is not None, 'Укажите ID файла GDrive'
    assert dataset_path is not None, 'Укажите путь к папке в которую сохранить датасет'
    
    destination = dataset_path + f'{ds_type}.zip'

    if not os.path.exists(destination):
        print(f'Загружаем {ds_type}.zip...')
        download_file_from_google_drive(gdrive_file_id, destination)
        
def extract_dataset(dataset_path, dataset_type):
    destination = dataset_path + f'{dataset_type}.zip'
    zip_ref = zipfile.ZipFile(destination, 'r')
    zip_ref.extractall(dataset_path)
    zip_ref.close()
    print('Загрузка завершена!\n')

In [20]:
# Загрузим наш датасет
# Так как с официальных ссылок Wider Face автоматически скачать не получается я перезалил их в облако
dataset_ids = {
    'train': '1sc3lUr_7LppHPEC5FhXiTIxn1HuhZNoo',
    'val': '1YW2_XQ1l30usbKkWCSrmf4oIC9AHRyga',
    'annotations': '1vgCABX1JI3NGBzsHxwBXlmRjaLV3NIsG'
}

!mkdir -p {DATASET_DIR}train
!mkdir -p {DATASET_DIR}val

for dataset_type in ['train', 'val', 'annotations']:
    dataset_loader(dataset_ids[dataset_type], DATASET_DIR, dataset_type)
    extract_dataset(DATASET_DIR, dataset_type)    

Загружаем train.zip...
Загрузка завершена!

Загружаем val.zip...
Загрузка завершена!

Загружаем annotations.zip...
Загрузка завершена!



In [21]:
# Посмотрим на загруженные данные
!ls -la {DATASET_DIR}

total 1789936
drwxr-xr-x 7 root root       4096 Sep 23 21:26 .
drwxr-xr-x 3 root root       4096 Sep 23 21:23 ..
drwxr-xr-x 3 root root       4096 Sep 23 21:25 WIDER_train
drwxr-xr-x 3 root root       4096 Sep 23 21:26 WIDER_val
-rw-r--r-- 1 root root    4494280 Sep 23 21:26 annotations.zip
drwxr-xr-x 2 root root       4096 Sep 23 21:26 test
drwxr-xr-x 2 root root       4096 Sep 23 21:26 train
-rw-r--r-- 1 root root 1465602149 Sep 23 21:25 train.zip
drwxr-xr-x 2 root root       4096 Sep 23 21:26 val
-rw-r--r-- 1 root root  362752168 Sep 23 21:26 val.zip


In [22]:
# Приведем к нормальному виду структуру папок
!mv {DATASET_DIR}WIDER_train/images {DATASET_DIR}train/images
!rm -R {DATASET_DIR}WIDER_train
!mv {DATASET_DIR}WIDER_val/images {DATASET_DIR}val/images
!rm -R {DATASET_DIR}WIDER_val

In [23]:
# Преобразуем данные в TFRecord для обучения
!python {DATA_DIR}convert_train_tfrecord.py --dataset_path="{DATASET_DIR}train" --output_path="{DATA_DIR}widerface_train_bin.tfrecord" --is_binary=True

2021-09-23 21:26:37.237470: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-23 21:26:37.244073: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-23 21:26:37.244796: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
I0923 21:26:37.245493 140703016339264 convert_train_tfrecord.py:132] Loading /app/retinaface/data/widerface/train
I0923 21:26:37.245643 140703016339264 convert_train_tfrecord.py:134] Reading data list...
I0923 21:26:38.239450 140703016339264 convert_train_tfrecord.py:144] Writing 12880 sample to tfrecord file...
  0%|                     

In [3]:
# Зададим настройки обучения модели
BATCH_SIZE = 4 # при большем размере GTX 1060 уходит в OOM
EPOCHS = 150

## Настройки конфигов обученияr
MODELS_CONFIG = {
    'ResNet50': {
        'cfg_path': f'{APP_DIR}configs/retinaface_res50.yaml',
        'dataset_path': f'{APP_DIR}data/widerface_train_bin.tfrecord',
        'batch_size': BATCH_SIZE,
        'testing_dataset_path': f'{APP_DIR}data/widerface/val',
        'epoch': EPOCHS
    },
    'MobileNetV2': {
        'cfg_path': f'{APP_DIR}configs/retinaface_mbv2.yaml',
        'dataset_path': f'{APP_DIR}data/widerface_train_bin.tfrecord',
        'batch_size': BATCH_SIZE,
        'testing_dataset_path': f'{APP_DIR}data/widerface/val',
        'epoch': EPOCHS
    }
}

# Выберем backbone для RetinaFace
backbone_model = 'ResNet50'

dataset_path = MODELS_CONFIG[backbone_model]['dataset_path']
cfg_path = MODELS_CONFIG[backbone_model]['cfg_path']
batch_size = MODELS_CONFIG[backbone_model]['batch_size'] 
testing_dataset_path = MODELS_CONFIG[backbone_model]['testing_dataset_path']
epoch = MODELS_CONFIG[backbone_model]['epoch']

custom_config_path = f'{APP_DIR}configs/my_train_cfg.yaml'

In [64]:
# Внесем в конфиг некоторые изменения
import re

print('Вносим изменения в конфигурационный файл...')

with open(cfg_path) as f:
    s = f.read()
with open(custom_config_path, 'w') as f:

    # Batch_size
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Epochs
    s = re.sub('^epoch: [0-9]+',
               'epoch: {}'.format(epoch), s)
    
    # Dataset path
    s = re.sub('dataset_path: \'.\/data\/widerface_train_bin.tfrecord\'', f"dataset_path: '{dataset_path}'", s)
    
    # Val path
    s = re.sub('testing_dataset_path: \'.\/data\/widerface\/val\'', f"testing_dataset_path: '{testing_dataset_path}'", s)
            
    f.write(s)
    
print(f'Создан новый конфигурационный файл {custom_config_path}')

Вносим изменения в конфигурационный файл...
Создан новый конфигурационный файл /app/retinaface/configs/my_train_cfg.yaml


In [65]:
!cat {custom_config_path}

# general setting
batch_size: 4
input_size: 640
backbone_type: 'ResNet50'  # 'ResNet50', 'MobileNetV2'
sub_name: 'retinaface_res50'

# training dataset
dataset_path: '/app/retinaface/data/widerface_train_bin.tfrecord'
dataset_len: 12880  # number of training samples
using_bin: True
using_flip: True
using_distort: True

# testing dataset
testing_dataset_path: '/app/retinaface/data/widerface/val'

# network
out_channel: 256

# anchor setting
min_sizes: [[16, 32], [64, 128], [256, 512]]
steps: [8, 16, 32]
match_thresh: 0.45
ignore_thresh: 0.3
variances: [0.1, 0.2]
clip: False

# training setting
epoch: 100
init_lr: !!float 1e-2
lr_decay_epoch: [50, 68]
lr_rate: 0.1
warmup_epoch: 5
min_lr: !!float 1e-3

weights_decay: !!float 5e-4
momentum: 0.9

pretrain: True

save_steps: 2000


## Обучаем модель

In [66]:
# Обучим нашу модель
!python {APP_DIR}train.py --cfg_path="{custom_config_path}" --gpu=0

2021-09-23 23:08:49.280789: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-23 23:08:49.287430: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-23 23:08:49.287940: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-23 23:08:49.288707: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

2021-09-23 23:09:07.025629: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.05GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-09-23 23:09:07.041118: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.08GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-09-23 23:09:07.274651: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.08GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-09-23 23:09:07.288940: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Alloc

Training [>>>>>>>>>>>>>>>>>>       ] 2420/3220, epoch=40/100, loss=0.2933, lr=1.0e-02  1.9 step/sec
[*] save ckpt file at ./checkpoints/retinaface_res50/ckpt-64
Training [>>>>>>>>>>>>>>>>>>>>>>>>>] 3220/3220, epoch=40/100, loss=0.5828, lr=1.0e-02  1.9 step/sec
Training [>>>>>>>>>                ] 1200/3220, epoch=41/100, loss=0.6364, lr=1.0e-02  1.9 step/sec
[*] save ckpt file at ./checkpoints/retinaface_res50/ckpt-65
Training [>>>>>>>>>>>>>>>>>>>>>>>> ] 3200/3220, epoch=41/100, loss=0.3134, lr=1.0e-02  1.9 step/sec
[*] save ckpt file at ./checkpoints/retinaface_res50/ckpt-66
Training [>>>>>>>>>>>>>>>>>>>>>>>>>] 3220/3220, epoch=41/100, loss=0.5075, lr=1.0e-02  1.9 step/sec
Training [>>>>>>>>>>>>>>>          ] 1980/3220, epoch=42/100, loss=0.3183, lr=1.0e-02  2.0 step/sec
[*] save ckpt file at ./checkpoints/retinaface_res50/ckpt-67
Training [>>>>>>>>>>>>>>>>>>>>>>>>>] 3220/3220, epoch=42/100, loss=0.7080, lr=1.0e-02  2.0 step/sec
Training [>>>>>                    ] 760/3220, epoch=43/

Training [>>>>>>>>>>>>>>>>>>       ] 2360/3220, epoch=63/100, loss=0.7001, lr=1.0e-03  1.9 step/sec
[*] save ckpt file at ./checkpoints/retinaface_res50/ckpt-101
Training [>>>>>>>>>>>>>>>>>>>>>>>>>] 3220/3220, epoch=63/100, loss=0.4830, lr=1.0e-03  1.9 step/sec
Training [>>>>>>>>                 ] 1140/3220, epoch=64/100, loss=0.2538, lr=1.0e-03  1.9 step/sec
[*] save ckpt file at ./checkpoints/retinaface_res50/ckpt-102
Training [>>>>>>>>>>>>>>>>>>>>>>>> ] 3140/3220, epoch=64/100, loss=0.1439, lr=1.0e-03  1.9 step/sec
[*] save ckpt file at ./checkpoints/retinaface_res50/ckpt-103
Training [>>>>>>>>>>>>>>>>>>>>>>>>>] 3220/3220, epoch=64/100, loss=0.5840, lr=1.0e-03  1.9 step/sec
Training [>>>>>>>>>>>>>>           ] 1920/3220, epoch=65/100, loss=0.2434, lr=1.0e-03  1.9 step/sec
[*] save ckpt file at ./checkpoints/retinaface_res50/ckpt-104
Training [>>>>>>>>>>>>>>>>>>>>>>>>>] 3220/3220, epoch=65/100, loss=0.2378, lr=1.0e-03  1.9 step/sec
Training [>>>>>                    ] 700/3220, epoch

# Тестируем модель

In [4]:
# Сформируем предсказания для проверки точности на валидационном датасете
!python {APP_DIR}test_widerface.py --cfg_path="/app/retinaface/configs/my_train_cfg.yaml" --gpu=0

2021-09-26 12:52:20.382186: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-26 12:52:20.447770: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-26 12:52:20.448443: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-26 12:52:20.449833: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

 [51 / 3226] det /app/retinaface/data/widerface/val/images/0--Parade/0_Parade_marchingband_1_78.jpg
 [52 / 3226] det /app/retinaface/data/widerface/val/images/0--Parade/0_Parade_Parade_0_247.jpg
 [53 / 3226] det /app/retinaface/data/widerface/val/images/0--Parade/0_Parade_marchingband_1_404.jpg
 [54 / 3226] det /app/retinaface/data/widerface/val/images/0--Parade/0_Parade_marchingband_1_606.jpg
 [55 / 3226] det /app/retinaface/data/widerface/val/images/0--Parade/0_Parade_Parade_0_664.jpg
 [56 / 3226] det /app/retinaface/data/widerface/val/images/0--Parade/0_Parade_marchingband_1_382.jpg
 [57 / 3226] det /app/retinaface/data/widerface/val/images/0--Parade/0_Parade_Parade_0_616.jpg
 [58 / 3226] det /app/retinaface/data/widerface/val/images/0--Parade/0_Parade_marchingband_1_149.jpg
 [59 / 3226] det /app/retinaface/data/widerface/val/images/0--Parade/0_Parade_Parade_0_873.jpg
 [60 / 3226] det /app/retinaface/data/widerface/val/images/0--Parade/0_Parade_marchingband_1_156.jpg
 [61 / 3226] de

 [133 / 3226] det /app/retinaface/data/widerface/val/images/1--Handshaking/1_Handshaking_Handshaking_1_236.jpg
 [134 / 3226] det /app/retinaface/data/widerface/val/images/1--Handshaking/1_Handshaking_Handshaking_1_107.jpg
 [135 / 3226] det /app/retinaface/data/widerface/val/images/1--Handshaking/1_Handshaking_Handshaking_1_827.jpg
 [136 / 3226] det /app/retinaface/data/widerface/val/images/1--Handshaking/1_Handshaking_Handshaking_1_579.jpg
 [137 / 3226] det /app/retinaface/data/widerface/val/images/1--Handshaking/1_Handshaking_Handshaking_1_275.jpg
 [138 / 3226] det /app/retinaface/data/widerface/val/images/1--Handshaking/1_Handshaking_Handshaking_1_35.jpg
 [139 / 3226] det /app/retinaface/data/widerface/val/images/1--Handshaking/1_Handshaking_Handshaking_1_411.jpg
 [140 / 3226] det /app/retinaface/data/widerface/val/images/1--Handshaking/1_Handshaking_Handshaking_1_158.jpg
 [141 / 3226] det /app/retinaface/data/widerface/val/images/1--Handshaking/1_Handshaking_Handshaking_1_209.jpg
 [

 [195 / 3226] det /app/retinaface/data/widerface/val/images/10--People_Marching/10_People_Marching_People_Marching_10_People_Marching_People_Marching_10_552.jpg
 [196 / 3226] det /app/retinaface/data/widerface/val/images/10--People_Marching/10_People_Marching_People_Marching_2_898.jpg
 [197 / 3226] det /app/retinaface/data/widerface/val/images/10--People_Marching/10_People_Marching_People_Marching_2_395.jpg
 [198 / 3226] det /app/retinaface/data/widerface/val/images/10--People_Marching/10_People_Marching_People_Marching_10_People_Marching_People_Marching_10_240.jpg
 [199 / 3226] det /app/retinaface/data/widerface/val/images/10--People_Marching/10_People_Marching_People_Marching_2_239.jpg
 [200 / 3226] det /app/retinaface/data/widerface/val/images/10--People_Marching/10_People_Marching_People_Marching_2_935.jpg
 [201 / 3226] det /app/retinaface/data/widerface/val/images/10--People_Marching/10_People_Marching_People_Marching_10_People_Marching_People_Marching_10_447.jpg
 [202 / 3226] det

 [261 / 3226] det /app/retinaface/data/widerface/val/images/12--Group/12_Group_Group_12_Group_Group_12_935.jpg
 [262 / 3226] det /app/retinaface/data/widerface/val/images/12--Group/12_Group_Large_Group_12_Group_Large_Group_12_650.jpg
 [263 / 3226] det /app/retinaface/data/widerface/val/images/12--Group/12_Group_Large_Group_12_Group_Large_Group_12_1007.jpg
 [264 / 3226] det /app/retinaface/data/widerface/val/images/12--Group/12_Group_Group_12_Group_Group_12_59.jpg
 [265 / 3226] det /app/retinaface/data/widerface/val/images/12--Group/12_Group_Team_Organized_Group_12_Group_Team_Organized_Group_12_171.jpg
 [266 / 3226] det /app/retinaface/data/widerface/val/images/12--Group/12_Group_Group_12_Group_Group_12_247.jpg
 [267 / 3226] det /app/retinaface/data/widerface/val/images/12--Group/12_Group_Team_Organized_Group_12_Group_Team_Organized_Group_12_283.jpg
 [268 / 3226] det /app/retinaface/data/widerface/val/images/12--Group/12_Group_Group_12_Group_Group_12_268.jpg
 [269 / 3226] det /app/retin

...

 [2916 / 3226] det /app/retinaface/data/widerface/val/images/57--Angler/57_Angler_peoplefishing_57_53.jpg
 [2917 / 3226] det /app/retinaface/data/widerface/val/images/57--Angler/57_Angler_peoplefishing_57_139.jpg
 [2918 / 3226] det /app/retinaface/data/widerface/val/images/57--Angler/57_Angler_peoplefishing_57_933.jpg
 [2919 / 3226] det /app/retinaface/data/widerface/val/images/57--Angler/57_Angler_peoplefishing_57_411.jpg
 [2920 / 3226] det /app/retinaface/data/widerface/val/images/57--Angler/57_Angler_peoplefishing_57_600.jpg
 [2921 / 3226] det /app/retinaface/data/widerface/val/images/57--Angler/57_Angler_peoplefishing_57_764.jpg
 [2922 / 3226] det /app/retinaface/data/widerface/val/images/57--Angler/57_Angler_peoplefishing_57_866.jpg
 [2923 / 3226] det /app/retinaface/data/widerface/val/images/57--Angler/57_Angler_peoplefishing_57_442.jpg
 [2924 / 3226] det /app/retinaface/data/widerface/val/images/57--Angler/57_Angler_peoplefishing_57_1012.jpg
 [2925 / 3226] det /app/retinaface/da

 [2988 / 3226] det /app/retinaface/data/widerface/val/images/59--people--driving--car/59_peopledrivingcar_peopledrivingcar_59_1019.jpg
 [2989 / 3226] det /app/retinaface/data/widerface/val/images/59--people--driving--car/59_peopledrivingcar_peopledrivingcar_59_85.jpg
 [2990 / 3226] det /app/retinaface/data/widerface/val/images/59--people--driving--car/59_peopledrivingcar_peopledrivingcar_59_117.jpg
 [2991 / 3226] det /app/retinaface/data/widerface/val/images/59--people--driving--car/59_peopledrivingcar_peopledrivingcar_59_202.jpg
 [2992 / 3226] det /app/retinaface/data/widerface/val/images/59--people--driving--car/59_peopledrivingcar_peopledrivingcar_59_1038.jpg
 [2993 / 3226] det /app/retinaface/data/widerface/val/images/59--people--driving--car/59_peopledrivingcar_peopledrivingcar_59_357.jpg
 [2994 / 3226] det /app/retinaface/data/widerface/val/images/59--people--driving--car/59_peopledrivingcar_peopledrivingcar_59_659.jpg
 [2995 / 3226] det /app/retinaface/data/widerface/val/images/

 [3062 / 3226] det /app/retinaface/data/widerface/val/images/61--Street_Battle/61_Street_Battle_streetfight_61_395.jpg
 [3063 / 3226] det /app/retinaface/data/widerface/val/images/61--Street_Battle/61_Street_Battle_streetfight_61_703.jpg
 [3064 / 3226] det /app/retinaface/data/widerface/val/images/61--Street_Battle/61_Street_Battle_streetfight_61_815.jpg
 [3065 / 3226] det /app/retinaface/data/widerface/val/images/61--Street_Battle/61_Street_Battle_streetfight_61_212.jpg
 [3066 / 3226] det /app/retinaface/data/widerface/val/images/61--Street_Battle/61_Street_Battle_streetfight_61_913.jpg
 [3067 / 3226] det /app/retinaface/data/widerface/val/images/61--Street_Battle/61_Street_Battle_streetfight_61_12.jpg
 [3068 / 3226] det /app/retinaface/data/widerface/val/images/61--Street_Battle/61_Street_Battle_streetfight_61_407.jpg
 [3069 / 3226] det /app/retinaface/data/widerface/val/images/61--Street_Battle/61_Street_Battle_streetfight_61_430.jpg
 [3070 / 3226] det /app/retinaface/data/widerface

 [3136 / 3226] det /app/retinaface/data/widerface/val/images/8--Election_Campain/8_Election_Campain_Election_Campaign_8_86.jpg
 [3137 / 3226] det /app/retinaface/data/widerface/val/images/8--Election_Campain/8_Election_Campain_Election_Campaign_8_252.jpg
 [3138 / 3226] det /app/retinaface/data/widerface/val/images/8--Election_Campain/8_Election_Campain_Election_Campaign_8_218.jpg
 [3139 / 3226] det /app/retinaface/data/widerface/val/images/8--Election_Campain/8_Election_Campain_Election_Campaign_8_297.jpg
 [3140 / 3226] det /app/retinaface/data/widerface/val/images/8--Election_Campain/8_Election_Campain_Election_Campaign_8_357.jpg
 [3141 / 3226] det /app/retinaface/data/widerface/val/images/8--Election_Campain/8_Election_Campain_Election_Campaign_8_529.jpg
 [3142 / 3226] det /app/retinaface/data/widerface/val/images/8--Election_Campain/8_Election_Campain_Election_Campaign_8_32.jpg
 [3143 / 3226] det /app/retinaface/data/widerface/val/images/8--Election_Campain/8_Election_Campain_Electi

 [3201 / 3226] det /app/retinaface/data/widerface/val/images/9--Press_Conference/9_Press_Conference_Press_Conference_9_196.jpg
 [3202 / 3226] det /app/retinaface/data/widerface/val/images/9--Press_Conference/9_Press_Conference_Press_Conference_9_767.jpg
 [3203 / 3226] det /app/retinaface/data/widerface/val/images/9--Press_Conference/9_Press_Conference_Press_Conference_9_552.jpg
 [3204 / 3226] det /app/retinaface/data/widerface/val/images/9--Press_Conference/9_Press_Conference_Press_Conference_9_100.jpg
 [3205 / 3226] det /app/retinaface/data/widerface/val/images/9--Press_Conference/9_Press_Conference_Press_Conference_9_595.jpg
 [3206 / 3226] det /app/retinaface/data/widerface/val/images/9--Press_Conference/9_Press_Conference_Press_Conference_9_43.jpg
 [3207 / 3226] det /app/retinaface/data/widerface/val/images/9--Press_Conference/9_Press_Conference_Press_Conference_9_828.jpg
 [3208 / 3226] det /app/retinaface/data/widerface/val/images/9--Press_Conference/9_Press_Conference_Press_Confer

In [10]:
# Запустим скрипт проверки точности наших предсказаний
!python {APP_DIR}widerface_evaluate/evaluation.py -p /app/widerface_evaluate/widerface_txt -g /app/retinaface/widerface_evaluate/ground_truth/

Processing hard: 100%|██████████████████████████| 61/61 [00:19<00:00,  3.20it/s]
==================== Results ====================
Easy   Val AP: 0.9326500330889518
Medium Val AP: 0.9272330939112695
Hard   Val AP: 0.8064947436317331


Мы получили достаточно неплохой результат для обучения при batch=4 на __GTX1060 6Gb RAM__. Оригинальную модель обучали на четырех NVIDIA Tesla P40 (24GB) GPUs с __batch_size=8x4__ в течении 80 эпох и результат на __Hard__ получили __0.914__.

__Easy   Val AP: 0.9326500330889518__

__Medium Val AP: 0.9272330939112695__

__Hard   Val AP: 0.8064947436317331__

![](imgs/retinaface_paper.png)

RetinaFace до сих пор является одним из sota решений в области детектирования лица.